In [42]:
import datetime
import json

# Load accounts.json
with open('../output/accounts.json', 'r', encoding='utf-8') as accounts_file:
    account_data = json.load(accounts_file)

# Load creation_dates.json
with open('../output/creation_dates.json', 'r', encoding='utf-8') as creation_dates_file:
    creation_dates = json.load(creation_dates_file)


date_str = account_data.get('date')
if date_str:
    date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
else:
    date = datetime.datetime.now()
accounts = account_data.get('accounts', [])

# augment accounts with creation dates
for account in accounts:
    profile_url = account.get('profile_url')
    if profile_url in creation_dates:
        account['creation_date'] = creation_dates[profile_url]
    else:
        account['creation_date'] = None

In [45]:
# Compute number of days between creation date and account_data date
import datetime

for account in accounts:
    creation_date = account.get('creation_date')
    if creation_date:
        creation_date = datetime.datetime.strptime(creation_date, '%Y-%m-%dT%H:%M:%S')
        days_since_creation = (date - creation_date).days
        account['days_since_creation'] = days_since_creation
    else:
        account['days_since_creation'] = None

# Compute follower growth rate (both account.stats.followers and days since creation can be None)
# add follower_growth_rate to each account's `stats` dictionary
for account in accounts:
    stats = account.get('stats', {})
    followers = stats.get('followers')
    days_since_creation = account.get('days_since_creation')
    
    if followers is not None and days_since_creation is not None and days_since_creation > 0:
        stats['follower_growth_rate'] = followers / days_since_creation
    else:
        stats['follower_growth_rate'] = None

result = {
    'date': date.strftime('%Y-%m-%d'),
    'accounts': accounts
}

# Save the updated accounts to a new JSON file
with open('../output/augmented_accounts.json', 'w', encoding='utf-8') as output_file:
    json.dump(result, output_file, ensure_ascii=False, indent=4)

Generate markdown report in `reports\accounts-top-25-growth.md`

In [47]:
# Prepare the markdown table header
markdown_table =  "| Account Name | Creation Date | Follower Count | Days since Created | Followers / Day |\n"
markdown_table += "|--------------|---------------|---------------:|-------------------:|----------------:|\n"

# Order the accounts by follower growth rate and limit to the top 25
# Filter out accounts with no follower growth rate
# Limit to the top 25 accounts
ordered_accounts = sorted(
    (account for account in accounts if account.get('stats', {}).get('follower_growth_rate') is not None),
    key=lambda x: x['stats']['follower_growth_rate'],
    reverse=True
)[:25]

# Populate the table rows
for account in ordered_accounts:
    # Skip accounts with no creation date or follower count
    if not account.get('creation_date') or not account.get('stats', {}).get('followers'):
        continue

    name = account['name']
    creation_date = account['creation_date'][:10] if account['creation_date'] else "N/A"
    followers = account['stats'].get('followers', 0)
    followers_formatted = f"{followers:,}"
    followers_per_day = account['stats'].get('follower_growth_rate', 0)
    followers_per_day_formatted = f"{int(followers_per_day):,}" if followers_per_day else "N/A"
    days_since_creation = account['days_since_creation'] if account['days_since_creation'] is not None else "N/A"
    days_since_creation_formatted = f"{int(days_since_creation):,}" if isinstance(days_since_creation, int) else "N/A"
    markdown_table += f"| {name} | {creation_date} | {followers_formatted} | {days_since_creation_formatted} | {followers_per_day_formatted} |\n"

# Print the markdown table
print(markdown_table)

# Save the markdown table to a file
with open('../reports/accounts-top-25-growth.md', 'w', encoding='utf-8') as md_file:
    # Write the header and the table to the markdown file
    md_file.write("# Augmented Accounts Report\n")
    md_file.write(f"**Acquisition date:** {date.strftime('%Y-%m-%d')}\n\n")
    
    md_file.write("## Top 25 accounts by follower growth rate\n")
    md_file.write("This table shows the top 25 accounts by follower growth rate, along with their creation date and follower count.\n\n")
    md_file.write(markdown_table)

    md_file.write("## Summary\n")
    md_file.write(f"**Total Accounts Processed:** {len(accounts)}\n\n")
    no_creation_date_count = sum(1 for account in accounts if not account.get('creation_date'))
    md_file.write(f"**Accounts with No Creation Date:** {no_creation_date_count}\n\n")
    no_follower_count = sum(1 for account in accounts if not account.get('stats', {}).get('followers'))
    md_file.write(f"**Accounts with No Follower Count:** {no_follower_count}\n\n")


| Account Name | Creation Date | Follower Count | Days since Created | Followers / Day |
|--------------|---------------|---------------:|-------------------:|----------------:|
| Telepress | 2025-03-30 | 477,000 | 28 | 17,035 |
| Citește | 2024-12-15 | 472,000 | 133 | 3,548 |
| România Eternă | 2025-03-13 | 116,000 | 45 | 2,577 |
| România Modernă | 2025-03-12 | 115,000 | 46 | 2,500 |
| România Autentică | 2025-02-09 | 132,000 | 77 | 1,714 |
| România Creștină | 2025-02-23 | 108,000 | 63 | 1,714 |
| Românesc | 2024-12-22 | 216,000 | 126 | 1,714 |
| Romania Mare | 2025-02-23 | 106,000 | 63 | 1,682 |
| România solidară | 2025-02-04 | 134,000 | 82 | 1,634 |
| Romania Culturala | 2025-01-20 | 143,000 | 97 | 1,474 |
| România, țara ta | 2025-02-07 | 116,000 | 79 | 1,468 |
| Original Românesc | 2025-02-20 | 94,000 | 66 | 1,424 |
| Vise Romanesti | 2025-01-18 | 123,000 | 99 | 1,242 |
| Sanatate curata | 2024-11-18 | 197,000 | 160 | 1,231 |
| Iubesc Romania | 2025-01-19 | 119,000 | 98 | 1,214